# Importing Necessary Libraries

In [1]:
import pandas as pd
import requests
import csv
import os
import zipfile
import io
import glob

# Fetching Fyers Data 

In [2]:
fyers_url = "https://public.fyers.in/sym_details/NSE_FO.csv"
fyers_df = pd.read_csv(fyers_url)
fyers_columns = [
    'Instrument_ID', 'Instrument_Name', 'Segment_ID', 'Lot_Size', 'Tick_Size',
    'Unused_Column', 'Trading_Hours', 'Expiry_Date', 'Timestamp', 
    'Exchange_Symbol', 'Precision', 'Multiplier', 'Unique_Code', 
    'Underlying_Asset', 'Asset_Code', 'Price_Change', 'Settlement_Type', 
    'Trading_Code', 'Option_Type', 'Strike_Price', 'Open_Interest'
]
fyers_df.columns = fyers_columns
fyers_df['Expiry_Date'] = pd.to_datetime(fyers_df['Expiry_Date'], errors='coerce')

# Parse 'Instrument_Name' for component breakdown

In [3]:
split_instrument = fyers_df['Instrument_Name'].str.split(' ', expand=True)
fyers_df[['Underlying_Asset', 'Expiry_Day', 'Expiry_Month', 'Expiry_Year']] = split_instrument[[0, 1, 2, 3]]
fyers_df['Strike_Price'] = pd.to_numeric(split_instrument[4], errors='coerce')
fyers_df['Option_Type'] = split_instrument[5]
fyers_df['Expiry_Date'] = pd.to_datetime(
    fyers_df['Expiry_Day'] + ' ' + fyers_df['Expiry_Month'] + ' ' + fyers_df['Expiry_Year'], errors='coerce'
)

# Dropping Unnecessary Columns and Saving File

In [4]:
drop_cols = ['Unused_Column', 'Timestamp', 'Precision', 'Multiplier', 'Trading_Code', 'Asset_Code', 'Price_Change', 'Settlement_Type']
fyers_df = fyers_df.drop(columns=drop_cols)
fyers_df.to_csv('fyers.csv', index=False)

# Fetching and saving Angel One data

In [5]:
angel_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
angel_response = requests.get(angel_url)
angel_data = angel_response.json()
angel_df = pd.DataFrame(angel_data)
angel_df.to_csv('angel_one.csv', index=False)

# ICICI data handling and extraction

In [6]:
icici_url = "https://directlink.icicidirect.com/NewSecurityMaster/SecurityMaster.zip"
os.makedirs('icici', exist_ok=True)

In [7]:
def download_extract_zip(url, extract_path='icici'):
    response = requests.get(url)
    response.raise_for_status()
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(extract_path)
        for txt_file in [f for f in z.namelist() if f.endswith('.txt')]:
            with open(os.path.join(extract_path, txt_file), 'r') as file:
                data = file.read().replace('|', ',')
            with open(os.path.join(extract_path, txt_file.replace('.txt', '.csv')), 'w') as csv_file:
                csv_file.write(data)

download_extract_zip(icici_url)

# Compile ICICI CSV files into a single DataFrame

In [8]:
csv_files = glob.glob('icici/*.csv')
icici_data_frames = [pd.read_csv(f, on_bad_lines='skip') for f in csv_files]
icici_df = pd.concat(icici_data_frames, ignore_index=True).drop_duplicates(subset=['Token']).fillna('')
icici_df.to_csv('icici.csv', index=False)

# Fetching kotak data and merging cash and futures data

In [9]:
kotak_urls = {
    "cash": "https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_Cash_01_04_2022.txt",
    "futures": "https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_01_04_2022.txt"
}

os.makedirs('kotak', exist_ok=True)
for filename, url in kotak_urls.items():
    response = requests.get(url)
    response.raise_for_status()  
    csv_data = []
    for line in response.text.strip().splitlines():
        csv_line = [element.strip() for element in line.split('|')]
        csv_data.append(','.join(csv_line))
    csv_content = '\n'.join(csv_data)

    path = f'kotak/{filename}.csv' 
    
    with open(path, 'w') as file:
        file.write(csv_content)

In [10]:
df_cash = pd.read_csv('kotak/cash.csv', on_bad_lines='skip')
df_futures = pd.read_csv('kotak/futures.csv', on_bad_lines='skip')

df_cash = df_cash.rename(columns={'OptionType': 'optionType'})

In [11]:
if set(df_cash.columns) == set(df_futures.columns):
    merged_data = pd.concat([df_cash, df_futures], ignore_index=True)
    print(merged_data.head()) 
else:
    print("The DataFrames do not have the same columns, cannot merge.")

   instrumentToken instrumentName                  name  lastPrice expiry  \
0             9516      20MICRONS        20 Microns Ltd      73.85      0   
1            43865      3IINFOLTD       3i Infotech Ltd      51.25      0   
2              891        3MINDIA          3M India Ltd   19697.30      0   
3              607         3PLAND  3P Land Holdings Ltd      14.50      0   
4            13140         5PAISA    5Paisa Capital Ltd     344.70      0   

   strike  tickSize  lotSize instrumentType segment exchange          isin  \
0     0.0      0.05        1             EQ    CASH      NSE  INE144J01027   
1     0.0      0.05        1             EQ    CASH      NSE  INE748C01038   
2     0.0      0.05        1             EQ    CASH      NSE  INE470A01017   
3     0.0      0.05        1             EQ    CASH      NSE  INE105C01023   
4     0.0      0.05        1             EQ    CASH      NSE  INE618L01018   

   multiplier  exchangeToken optionType  
0           1          169

In [12]:
merged_data.to_csv('kotak.csv', index=False)

In [13]:
df_kotak = pd.read_csv('kotak.csv',low_memory=False)
df_kotak.head()

,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,optionType
0,9516,20MICRONS,20 Microns Ltd,73.85,0,0.0,0.05,1,EQ,CASH,NSE,INE144J01027,1,16921,-
1,43865,3IINFOLTD,3i Infotech Ltd,51.25,0,0.0,0.05,1,EQ,CASH,NSE,INE748C01038,1,6232,-
2,891,3MINDIA,3M India Ltd,19697.30,0,0.0,0.05,1,EQ,CASH,NSE,INE470A01017,1,474,-
3,607,3PLAND,3P Land Holdings Ltd,14.50,0,0.0,0.05,1,EQ,CASH,NSE,INE105C01023,1,2595,-
4,13140,5PAISA,5Paisa Capital Ltd,344.70,0,0.0,0.05,1,EQ,CASH,NSE,INE618L01018,1,445,-


# Fetching Zerodha Data

In [14]:
zerodha_url = "https://api.kite.trade/instruments"
zerodha_file = "zerodha.csv"
response = requests.get(zerodha_url)
with open(zerodha_file, 'w', newline='') as file:
    writer = csv.writer(file)
    for line in response.text.splitlines():
        writer.writerow(line.split(','))

# Renaming and merging columns for unified dataset

In [15]:
rename_mapping = {
    'Fyers': {
        'Instrument_ID': 'token', 'Instrument_Name': 'name', 'Lot_Size': 'lotSize', 'Tick_Size': 'tick_size',
        'Expiry_Date': 'expiry', 'Option_Type': 'optionType', 'Strike_Price': 'strike'
    },
    'AngelOne': {
        'token': 'token', 'symbol': 'tradingsymbol', 'name': 'angel_name', 'expiry': 'expiry', 
        'strike': 'strike', 'lotsize': 'lotSize', 'instrumenttype': 'optionType'
    },
    'ICICI': {
        'Token': 'token', 'ShortName': 'tradingsymbol', 'InstrumentName': 'icici_name', 
        'ExpiryDate': 'expiry', 'StrikePrice': 'strike', 'LotSize': 'lotSize', 'OptionType': 'optionType'
    },
    'Kotak': {
        'instrumentToken': 'token', 'instrumentName': 'kotak_name', 'expiry': 'expiry', 
        'strike': 'strike', 'lotSize': 'lotSize', 'optionType': 'optionType'
    },
    'Zerodha': {
        'instrument_token': 'token', 'tradingsymbol': 'tradingsymbol', 'name': 'zerodha_name',  
        'expiry': 'expiry', 'strike': 'strike', 'lot_size': 'lotSize', 'instrument_type': 'optionType'
    }
}

# Load and rename columns for each DataFrame

In [16]:
fyers_df = pd.read_csv('fyers.csv').rename(columns=rename_mapping['Fyers'])
angel_df = pd.read_csv('angel_one.csv').rename(columns=rename_mapping['AngelOne'])
icici_df = pd.read_csv('icici.csv').rename(columns=rename_mapping['ICICI'])
kotak_df = pd.read_csv('kotak.csv').rename(columns=rename_mapping['Kotak'])
zerodha_df = pd.read_csv('zerodha.csv').rename(columns=rename_mapping['Zerodha'])

for df in [fyers_df, angel_df, icici_df, kotak_df, zerodha_df]:
    df['token'] = df['token'].astype(str)

C:\Users\Chetan\AppData\Local\Temp\ipykernel_25540\4075534302.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  angel_df = pd.read_csv('angel_one.csv').rename(columns=rename_mapping['AngelOne'])
C:\Users\Chetan\AppData\Local\Temp\ipykernel_25540\4075534302.py:3: DtypeWarning: Columns (0,1,2,3,11,12,25,33,34,35,37,66,86,95,96,97,104,154) have mixed types. Specify dtype option on import or set low_memory=False.
  icici_df = pd.read_csv('icici.csv').rename(columns=rename_mapping['ICICI'])
C:\Users\Chetan\AppData\Local\Temp\ipykernel_25540\4075534302.py:4: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  kotak_df = pd.read_csv('kotak.csv').rename(columns=rename_mapping['Kotak'])


# Merge all data sources

In [17]:
merged_df = pd.merge(fyers_df, angel_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer')
merged_df = pd.merge(merged_df, icici_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer')
merged_df = pd.merge(merged_df, kotak_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer')
merged_df = pd.merge(merged_df, zerodha_df, on=['token', 'expiry', 'strike', 'lotSize', 'optionType'], how='outer')

In [18]:
merged_df.to_csv('final_merged_data.csv', index=False)
print("Consolidation Complete!")

Consolidation Complete!
